In [3]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait

import csv
import json
import time
import urllib
import requests
import pandas as pd
import nltk
import spacy
import string
import numpy as np

In [56]:
df = pd.read_csv("movie_ids.csv", converters={'movie_ids': lambda x: str(x)})
ids = df["movie_ids"]
ids = list(df.iloc[:100]["movie_ids"])
len(ids)
ids[46]

'0110186'

In [51]:
chrome_options = webdriver.ChromeOptions(); 
chrome_options.add_experimental_option("excludeSwitches", ['enable-automation']);

In [52]:
reviews = []
ratings = []

In [53]:
def extract_ratings():
    rating_reviews = driver.find_elements_by_xpath(".//div[@class='review-container']")
    for rating_review in rating_reviews:
        try:
            new_rating = rating_review.find_element_by_xpath(".//span[@class='rating-other-user-rating']/span[1]")
            ratings.append(new_rating.text)
        except NoSuchElementException:  
            ratings.append(None)

In [54]:
def extract_reviews():
    rating_reviews = driver.find_elements_by_xpath(".//div[@class='review-container']")
    for rating_review in rating_reviews:
        review = rating_review.find_element_by_xpath(".//div[@class='content']/div[@class='text show-more__control']")
        reviews.append(review.text)

In [59]:
s = 0
sum = 0
for id in ids: 
    sum += 1
    url = "https://www.imdb.com/title/tt" + id + "/reviews"
    
    req = urllib.request.Request(url, headers={'User-Agent' : "Magic Browser"}) 

    driver = webdriver.Chrome(options=chrome_options)    

    driver.get(url)

    show_more_button = driver.find_elements_by_class_name("ipl-load-more__button")
    
    if show_more_button != []:
        while show_more_button[0].value_of_css_property("display") == "block":
            show_more_button[0].click() 
            time.sleep(3)

    read_full_buttons = driver.find_elements_by_xpath(".//div[@class='gradient-expander  show-more']")

    # Click on read more button to read
    non_spoiler =[]

    #  Non spoiler review
    for read_full_button in read_full_buttons:
        full_review = read_full_button.find_element_by_class_name("expander-icon-wrapper")

        non_spoiler.append(full_review)

    # Spoiler review
    spoiler = driver.find_elements_by_xpath(".//div[@class='expander-icon-wrapper spoiler-warning__control']")

    show_more_reviews = non_spoiler + spoiler

    for show_more_review in show_more_reviews:
        show_more_review.click()
        
    if sum == 20:
        s = s + 1
        dataframe = pd.DataFrame({"reviews": reviews, "ratings": ratings})
        dataframe.to_csv('movie_review-' + str(s) + '.csv')
        reviews = []
        ratings = []
        

    extract_ratings()
    extract_reviews()
    
    print(sum)
    driver.close()


In [223]:
dataframe = pd.DataFrame({"reviews": reviews, "ratings": ratings})

dataframe.to_csv('movie_review.csv')